# Mobile Robotics

Kevin J. Walchko
created 19 Oct 2017

----

Now we are going to start talking about mobile robotics (robots that move) as apposed to the robot arm we used in the beginning. We will cover some basic design steps and things to keep in mind when building a robot. This lesson will also have some of the same ideas from the kinematic (arm) lessons and you should recognize some familar things like rotation matricies and coordinate systems.

## Objectives

- understand mobile robot coordinate frames
- understand mobile robot kinematics in those frames
- understand general robotic control
- understand simple robot design for power and movement

## References

- Siegwart, R., Nourbakhsh, I., *Introduction to Autonomous Mobile Robots*
- [Euler angles](https://www.learnopencv.com/rotation-matrix-to-euler-angles/)
- [Wikipedia euler angles](https://en.wikipedia.org/wiki/Euler_angles#Tait.E2.80.93Bryan_angles)
- [Wikipedia Lead acid battery](https://en.wikipedia.org/wiki/Lead%E2%80%93acid_battery)
- [Wikipedia NiMH batteries](https://en.wikipedia.org/wiki/Nickel%E2%80%93metal_hydride_battery)
- [Wikipedia Li-ion batteries](https://en.wikipedia.org/wiki/Lithium-ion_battery)
- [Wikipedia Li-pi batteries](https://en.wikipedia.org/wiki/Lithium_polymer_battery)
- [Wikipedia LiFePO4 batteries](https://en.wikipedia.org/wiki/Lithium_iron_phosphate_battery)

In [1]:
%matplotlib inline 

import numpy as np # matrix manipulations
from matplotlib import pyplot as plt

# Asimov's Robot Laws

[![](pics/3-laws.png)](https://xkcd.com/1613/)

No robotics class would be complete without mentioning [Isaac Asimov's](https://en.wikipedia.org/wiki/Isaac_Asimov) (Biochemist who taught at Boston University) [Three Laws of Robotics](https://en.wikipedia.org/wiki/Three_Laws_of_Robotics):

1. A robot may not injure a human being or, through inaction, allow a human being to come to harm
1. A robot must obey the orders given it by human beings except where such orders would conflict with the First Law
1. A robot must protect its own existence as long as such protection does not conflict with the First or Second Laws

# Design Philosophy

Fueled by the introduction of microprocessors in the the 70's, early robotics focused on building complex human robots. Think about the state of sensors, programming languages, processing power, etc in those early years and you can begin to understand why there was so much failure. Eventually, engineers from MIT began to advocate building "simple" robust insect like robots with simple understandings, sensors, and control process. Honestly, how complex was your robot in ECE382?

## Understanding Your World

So following a biological path, how do we understand our world and exist in it? Most animals/insects have:

- sight: from complex human eyes, to dynamic nightvision eyes (cats), to simple grayscale light detectors on bugs
- hearing: here I would span the range from voice recognition to being able to decect a sound (hand clapping, see the 90's sensation, [the clapper](https://en.wikipedia.org/wiki/The_Clapper))
- touch: sensing another object is vital to navigation. This could be both physical pressure or LIDAR
- emotion: You need something to drive your robot to move or do something. Maybe *emotion* is a strong word, but it is its reason to exist and its purpose that drives it to continue
- hunger: eventually, everything needs to eat (recharge) or it dies

Many authors have described these with different words, or in different ways, but they are basically all the same. Generally you have to design a robot to do *something* and it will need to sense the world and recharge. This is the simple basis behind the successful Roomba vacuum.

# Simple Robot Design

![](pics/design_flow.png)

If you ever had to design your own robot from scratch and build it, the above **simple** diagram could help you get started. It assumes your inputs are battery chemistry, run-time, speed and the environment you are working in. Depending on your situation, you could have other inputs or one of the inputs, say speed, is totally unimportant so you let the design pick the speed of teh robot.

Agina, there are lots of aspect of robot design that can make this process difficult and complex. However, we will just look at some simple, but important aspects of robotic design that all Electrical and Computer Engineers should understand and be very comfortable with.

- Power system
- Propulsion
- Control

## Power Systems

Yes, you all took ECE231, so this shouldn't be too unfamilar to you. Generally, mobile robots (and other embedded systems like cell phones, satellites, etc) are powered by a battery. Let's talk about some of the trade-off or choices you have in robot design.

### Battery Types

Batteries are generally characterized by chemistry, voltage, and capacity. Realize that chemistry also determines a battery's density and hence it's size for a given capacity.

| Type | Key Point | Energy Density | Description |
|:-----|:----------|:---------------|:------------|
| NiMH | Safe | meduim | NiMh batteries are rechargeable and their value (price / capacity / weight) and availability is hard to beat. There is almost no memory effect, meaning every charge should bring the battery up to full capacity. They are safe, if you over charge them, they will dissipate the excess energy as heat (which is why they are warm coming off a charger)|
| NiCd | Environmentally bad | low | These batteries are slowly disappearing because of their memory effect: if you don’t discharge the battery properly and then recharge it to full capacity, you lose part of the capacity each time. Also, the cadium is really bad for the environment (very toxic)|
| Alkaline | non-rechargable, environmentally bad | high | These are the least expensive batteries in the short term, and provide a higher voltage than NiMh, but are not great for the environment (you must throw them away and they end up in a landfill somewhere), and you constantly need to buy replacements. |
| Lead Acid | Old, heavy, low discharge rates | low | Still the cheapest option for high capacity, lead acid is usually reserved for medium/large sized robots because of their incredibly high weight (remember it has lead in it ... heavy). Also, lead acid has life issues. Generally you can not fully discharge them and if they are not routinely charged, they can have [sulfation](https://en.wikipedia.org/wiki/Lead%E2%80%93acid_battery#Sulfation_and_desulfation) and loose capacity. |
| LiIon/LiPo | FIRE, small, light, come in many different shapes | high | These are fast becoming the most popular type of battery (especially in the RC toy world) because of their light weight, high discharge rates and relatively good capacity, except the voltages increase in increments of 3.7V, so you need to plan to use LiPo before selecting your electronics and actuators. They are also very dangerous and if mishandled, they will catch file. Samsung is famous for design errors in their Li-ion batteries and costing the company around $20B ... and Samsung has had decades of experience with these batteries. Most other companies (Google, Apple, Dell, etc) have also had issues. Be vary careful with these batteries. |
| LiFePO4 | Safer | medium-high | Created by MIT, these are much safer than other lithium batteries, but they do not have the energy density. However, because of the addition of Fe (iron, which is heavy) to the chemistry, if treated correctly, they are much safer than their lithium cousins (because the Fe has a much stronger bond to the oxygen molecules and less like likely to combust). |

### How Many Batteries

Let's talk about batteries.

- Multiple batteries
    - Pros
        - Can make your design simpiler
        - Batteries can be at different voltages, sizes, located close to where they are being used, etc
        - Reduce cabling harnes complexity and the need to split cables (i.e., Y harness)
        - Reduce electrical noise from motors to sensitive electronics, or reduce added hardware to clean up your power system to your electronics
    - Cons
        - You have to be able to access the multiple batteries and re-charge them
        - Added cost and weight
        - If batteries are different chemisties, then you may need different types of chargers
        - You could be faced with different run-times for different parts of your robot
        - You would have to track voltage status of multiple batteries to know when to recharge what battery
        - Battery spares become more complex, especially if battery sizes, voltage, chemestry, capcity, etc are different between the multiple batteries
- Single batter
  - Pros
    - Simple charging system
    - Entire system has only 1 run-time
    - Simple monitoring system because 1 battery
    - Few spare batteries needed for logistics
  - Cons
    - Complex wireing harness
    - Need buck/boost converters to change voltage levels to different sub-systems

### Discharge Rate (C rating)

Batteries are designed to charge and discharge at certain rates. Going above these rates can be dangerous or life limiting to the battery. Most batteries can discharge at 1C or less and be safe. Some can discharge at 2C or less. The C in this case is their capacity, so a 2000mAhr battery can discharge at 2000mA (or less) for 1 hr and be fine. There are some Li-ion batteries made for RC toys that are designed to discharge at 10C, 20C or 30C. **However, please understand they were designed for that and not all Li-ion batteries can do that.** You should always look at the **datesheet** to understand what a safe discharge rate is for your battery.

## Example 1

Assume we have a robot and need to figure out a battery for it.

- Power budget
  - electronics and sensors are negligible compared to motors
  - 2x Drive Motors: 6V nominal, 1A each under normal load
- Battery selection
  - Want rechargeable, safe, medium-density, high availability (meaning I can buy the batteries at Target in the checkout line)
  - 5x 1.2V AA NiMh Batteries, each providing 2200mAh (equivalent to 2.2Ah) when connected serially will provide 5x1.2V = 6V
  - Note that the battery was chosen based on the motor’s nominal voltage. 

Therefore the 6V battery pack will last: 2.2Ah battery / (2 motors x 1A per motor) = 1.1 hrs

## Example 2

Repeat above, but select a 7.4V 3000mAhr Li-ion battery, because we are not worried about fire.

So we are going to use a buck converter to lower the 7.4V to 6V for the motors. Our battery provides 7.4V x 3Ahr = 22.2W of power. The buck converter (assume it has no limitation and is 100\% efficient, will provide 22.2W/6V = 3.7Ahr.

Therefore the run-time is: 3.7Ahr / (2 motors x 1A per motor) = 1.85 hrs

Note: if the efficiency was something more realistic like 90\% for a buck converter, then the run-time would be: .9 x 3.7Ahr / (2 motors x 1A per motor) = 1.65 hrs. So efficiency hurts you, but if you can get 90-96\% efficient buck converters (which exist), then it isn't that bad.

## Example 3

Let's do a walking robot, hexipod (6 legs, 3 servos per leg)

- Power budget
  - 18 servos which operate at 6V nominal and consume 250mA under normal load
  - Sensors and electronics are negliable compared to motors
- Battery selection
  - Want rechargeable, safe, medium-density, high availability
  - 10x 1.2V AA NiMh batteries
    - 5 serially connected for 6V, 2.2Ahr
    - 2 of these connected in parallel for 5S2P configuration giving 6V, 4.4Ahr

First, we will assume that all motors are under load at all times (i.e. worst case scenario) and therefore all 18 will be consuming a total of 4.5A

4.4Ah battery / 4.5A = 0.98 hours

### Power System Reality

So in all of these cases, we assumed the motors were the largest current draw and electronics/sensors were negligible. That is not allways true, so in real life you would just have to add their power into the power budget and deal in power like we did with Example 2.

## Propulsion

Depending on the type of robot, there are many ways to make it move. We are going to break it into 2 types for ground robots (not flying): holonomic and non-holonomic motion. These choices will effect how you are able to control your robot.

## Non-holonomic

<img src="pics/robot-frame.png" width="300px">

- Constraints limit the possible movements
    - Looking at a robot like the Roomba, it has 2 wheels and a caster. The wheels can only move in the x-direction and not in the y-direction. The exception is during turning, the wheels will slip a little in the y-direction which allow you to turn quicker, but throws off your odometry.
- This is typically a velocity constraint.
    - The robot can occupy any point in a plane at any orientation, but it may not be able to directly transition from one point/orientation to another
    - Cars and parallel parking are a great example of this
    - Our Roomba we will use is also in this class of robot.

## Holonomic

- There are no constraints on your robot and sideways movement is possible
  - This is dependent on your wheel/propulsion setup. Shown below is the classic omni wheel and holonomic robot used in robot soccer (looking up from the bottom). It is important to notice that the wheel is designed to slip and slide. This is a blessing and a curse:
  <img src="pics/omni_wheel.jpg" width="200px"><img src="pics/holonomic_robot.png" width="200px">
   - Becuase the wheels are designed to slip and slide, non-holonomic robots have isses:
     - towing, no traction to pull
     - difficulty going up hills/inclines, same reason as towing
     - have to spin a lot of wheels, which slip, to get movement so they consume more power
     - more wheels, motors, drive trains, etc means more desing complexity and weight
     - Movement commands are more complex (involves matrix/vector to generate motor values), but once defined, it is easy. 
       - There is another Jupyter notebook in this folder that you can look at and see the math behind it. The details are best left to a Mechanical Engineer who is good at kinematics/dynamics and understands state space equations/control.
  <img src="pics/omni_wheel_control.png" width="400px">

# Notation

When we derive the equations of motion (EoM) for a robot, we will use notation like the following:

| | |
|---------------|--------------|
| $x$           | position     |
| $\dot x$      | velocity     |
| $\ddot x$     | acceleration |
| $\theta$      | heading (yaw)|
| $\dot \theta$ | heading rate |

where $x$ is often referred to as a state variable, although it could be any letter of the alphabet or a Greek mathematical symbol. If we take the time derivative of that variable with respect to time, we get velocity. If we take the derivative a second time, we get acceleration. Throughout, we will try to differentiate between a coordinate systems' x and y values and a robot's state variable.

Also, becaues we are working with the roomba, we will focus on nonholonomic systems. 

## Euler Angles

<img src="pics/body.png" width="400px">

Euler angles describe the moving frame and not the fixed (inertial) frame. They follow the sequence (see Craig eqn 2.70) which descibes the end-effector (or the aircraft pictured above) relative to the fixed frame:

- **Roll:** rotate about the x-axis
- **Pitch:** rotate about (the new) y-axis
- **Yaw:** rotate about (the new) z-axis

Please notice, each rotation is dependant on previous rotations. Also notice, you can tell which axis the rotation is around by looking for the **1** in the matrix.

\begin{eqnarray}
  R_x(\gamma) = \begin{bmatrix}
    1 & 0 & 0 \\
    0 & \cos(\gamma) & -\sin(\gamma) \\
    0 & \sin(\gamma) & \cos(\gamma)
  \end{bmatrix} \\
  R_y(\beta) = \begin{bmatrix}
    \cos(\beta) & 0 & \sin(sin(\beta) \\
    0 & 1 & 0 \\
    -\sin(\beta) & 0 & \cos(\beta)
  \end{bmatrix} \\
  R_z(\alpha) = \begin{bmatrix}
    \cos(\alpha) & -\sin(\alpha) & 0 \\
    \sin(\alpha) & \cos(\alpha) & 0 \\
    0 & 0 & 1
  \end{bmatrix} \\
  R^A_B = R_z(\alpha) R_y(\beta) R_x(\gamma)
\end{eqnarray}


Now, for this class, we are not going to use these right now, but you will see them pop-up in later lessons which is why I mention them. However, it is also easier to explain or talk about rotation matricies using Euler angles.

The last combined rotation $R_z(\alpha) R_y(\beta) R_x(\gamma)$ is also refered to as $R_{zyx}$ or $R_{321}$ where x is 1, y is 2, and z is 3. The later name is often seen in the aerospace community and is the sequence to go between a vehicles body coordinate system to an inertial refernces frame. If you deal with aircraft or satellites, you might see stuff like this. Besides the $R_{321}$, there are 11 other combinations of rotations that can be used. Different fields of study use different sequences based on where singularities occur.

Also, notice how each row and column has a sin and cos in it. If you take the magnitude of a row or column, it will always be 1.

# Coordinate Systems

It is important for robots to be able to travel from one place to another. In order to determine where the robot is, we typically asign 2 reference frames. A global frame (e.g. inertial frame) so we can understand the dynamics of the robot and a body frame to understand what the orientation of the robot is relative to the global frame. Also, remember in a previous lesson, we talked about **pose**, which is the position and orientation of something relative to aother frame.

<img src="pics/robot-frame.png" width="400px">

Another things to note is how the body frame is oriented. There is no standard way *generally*, however, most systems use the x-direciton of the body axis as forward. Then for aerospace tyically the y-axis goes out the right wing and z-axis points down. Others dislike the z-axis pointing down, so they orient the y-axis out the left side of the body and now the z-axis points up. There is no advantage or disadvantage with these, just personal preference.

For our work with the roomba, the only orientation we are concerned about is *yaw* or *heading* (about the z-axis) which is typically associated with a cardinal (or compass) direction. We will represent the robot's pose (or also known as *state*) as (see Siegwart, eqns 3.1 - 3.2):

$$
\xi =
\begin{bmatrix}
  x \\
  y \\
  \theta
\end{bmatrix} \\
R^I_B(\theta) =
\begin{bmatrix}
  \cos(\theta) & \sin(\theta) & 0 \\
  -\sin(\theta) & \cos(\theta) & 0 \\
  0 & 0 & 1
\end{bmatrix} \\
\xi_I = R^I_B(\theta) \xi_B \\
R^B_I(\theta) = inv(R^I_B(\theta)) = R^I_B(\theta)^T
$$

where subscript I refers to the inertial frame, subscript B refers to the body frame. Hopefully you also recognize this rotation matrix as a rotation about the z-axis by $\theta$. 

The speed of a wheel is a function of its wheel radius ($r$) and how fast the motor turns  the wheel is $\dot \phi$.

![](pics/wheel_vel.png)

$$
v_{wheel} = \dot \phi r
$$

Remeber, we always *assume* there is no slip when a wheel  moves. Thus at point A, the velocity of the wheel is 0. At point B, the velocity of the wheel is $\dot \phi r$ as shown above. While the velocity of the wheel at point C is $2V$ because you are now x2 the distance from A. This is mainly just an interesting mathematical (physics) thing.

Now using these ideas, we can write, the speed of the robot in the x-direction is equal to the $\dot \phi r$ assumming both wheels have the same radius (r). But, there are 2 wheels, each providing the same force if we are going straight forward. Thus each wheel provides half the velocity of the total robot.

We have already discussed the wheels of the robot cannot move in the y-direction, so it is 0.

Finally, if the wheels are moving in different directions, then the robot will spin in place. Thus the $\theta$ element is the difference of the two.

$$
\dot \xi_B = \begin{bmatrix}
  \frac{r \dot \phi_1}{2} + \frac{r \dot \phi_2}{2} \\
  0 \\
  \frac{r \dot \phi_1}{2} - \frac{r \dot \phi_2}{2}
\end{bmatrix} \\
\dot \xi_I = R^I_B(\theta) \dot \xi_B
$$

For example, substituting the following values into the previous equations gives:

- $\theta = \frac{\pi}{2}$
- r = 1
- l = 1
- $\dot \phi_1 = 4$
- $\dot \phi_2 = 2$

$$
\dot \xi_I = \begin{bmatrix}
  \dot x \\
  \dot y \\
  \dot \theta
\end{bmatrix} = 
\begin{bmatrix}
  0 & -1 & 0 \\
  1 & 0 & 0 \\
  0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
  3 \\
  0 \\
  1
\end{bmatrix} = 
\begin{bmatrix}
  0 \\
  3 \\
  1
\end{bmatrix}
$$

So what does this tell us? We know both motors are driving forward (positive velocity), but they are not going the same speed. Therefore we are turning a little. However, our body frame is not aligned with the inertial frame, so although we are moving forward in the x-dir of the body, we are actually moving forward in the inerital y-dir frame. This is navigation ... frames, frames, frames!

# Questions

1. What are the types of batteries discussed in this lesson and what are their pros/cons?
1. When designing a robot, what are the considerations that go into it? *Hint:* see understanding your world section and the awesome flow chart.
1. If I need a robot to run for 30 minutes and it consumes 5A, what would battery capacity would I need in mAhrs?
1. If I have a battery that provides 50W of power and I have 2 motors that operate at 5V, 2A each and a Raspberry Pi that operates at 5V, 2A ... how long can my robot run for?
1. How does efficiency effect your power calculations?
1. What are 2 important reference frames discussed in this lesson?
1. If my robot's body frame is angled 20 degrees from my inertial frame, what is the rotation matrix to convert body vectors to inertial vectors? What about the opposite, from inertial to body frame?
1. What is the difference between a holonomic and non-holonomic robot?
    1. Which type is the roomba we will use?
1. What is an issue with encoders you need to be aware of?
1. If my wheel has a radius of 5 inches and it is turning at 30 rpm, how fast is it going in mm/sec? *Hint:* there are some units conversion here.
1. For Euler angles, about which axes are roll, pitch, and yaw measured about?
1. Given an Euler rotation, how would you know if the rotation matrix is about a single axis?


-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.